In [1]:
import numpy as np
from scipy.stats import kendalltau
from scipy.stats import rankdata
from itertools import permutations
import pandas as pd

# Aggregating the results of different metrics (WIP)

## Kameny consensus

Each metrics will provide a ranking of different algorithms, that could be represented as a permutation of [1:N]. Insofar these rankings might be different according to the metrics, it is necessary to find a way to generate a new permutation that would be close to each of the permutations generated by the benchmark metrics. The first approach, called Kameny consensus, is based on a distance (the Kendall distance), that measures the dissimilarity between two permutations. 
Scipy as a module that computes a Kendall correlation (not to be confounded with a Kendall Tau distance).
Our first goal is to code a function to evaluate the Kendall distance between two permutations

In [2]:
kendalltau([1, 2, 3], [1, 2, 3])
# Not what we are looking for

KendalltauResult(correlation=1.0, pvalue=0.3333333333333333)

In [3]:
def is_negative(n):
    if n < 0:
        return 1
    return 0


def Kendall_distance(l1, l2):
    if len(l1) != len(l2):
        raise ValueError("Les permutations n'ont pas la même longueur")
    result = 0
    for i in range(len(l1)):
        for j in range(len(l1)):
            result += is_negative((l1[i] - l1[j]) * (l2[i] - l2[j]))
    normalization = len(l1) * (len(l1) - 1)
    # normalization is made make the distance be in [0,1]
    return result / normalization

In [4]:
print(Kendall_distance([0, 1, 2], [0, 1, 2]))
print(Kendall_distance([0, 1, 2], [2, 1, 0]))
print(Kendall_distance([0, 1, 2], [1, 0, 2]))

0.0
1.0
0.3333333333333333


In [5]:
def Kameny_consensus_step1(M, perm):
    # M is an array in which M[i,j] is the ranking of algorithm i at test j
    result = 0
    for i in range(M.shape[0]):
        result += Kendall_distance(perm, M[i])
    return result


def Kameny_consensus(M):
    L = []
    dist = Kameny_consensus_step1(M, M[0])
    perm = permutations(M[0])
    for permutation in list(perm):
        dist_travail = Kameny_consensus_step1(M, permutation)
        if dist_travail < dist:
            L = []
            L += [np.array(permutation)]
            dist = dist_travail
        elif dist == dist_travail:
            L += [np.array(permutation)]
    return L, dist

In [6]:
M = np.array([[3, 2, 1, 0], [1, 2, 3, 0], [1, 3, 2, 0], [0, 1, 3, 2]])
perm1 = [0, 1, 2, 3]
print(Kameny_consensus_step1(M, perm1))
perm2 = [3, 2, 1, 0]
print(Kameny_consensus_step1(M, perm2))

2.333333333333333
1.6666666666666665


In [7]:
Kameny_consensus(M)

([array([1, 3, 2, 0]), array([1, 2, 3, 0])], 1.0)

## First improvement of Kameny consensus

All the rankings, given the different metrics have exactly the same weights. This might be a problem if we consider that some of the metrics are more relevant than others, or if some metrics are usually very close. So we can improve the Kameny consensus by giving a higher weight to metrics that are more relevant

In [8]:
def Weighted_Gameny_consensus_step1(M, perm, Weights):
    # M is an array in which M[i,j] is the ranking of algorithm j at test i
    # Weights[i] is the weight associated to test i
    result = 0
    for i in range(M.shape[0]):
        result += Kendall_distance(perm, M[i]) * Weights[i]
    return result


def Weighted_Kameny_consensus(M, Weights):
    L = []
    dist = Weighted_Gameny_consensus_step1(M, M[0], Weights)
    perm = permutations(M[0])
    for permutation in list(perm):
        dist_travail = Weighted_Gameny_consensus_step1(M, permutation, Weights)
        if dist_travail < dist:
            L = []
            L += [np.array(permutation)]
            dist = dist_travail
        elif dist == dist_travail:
            L += [np.array(permutation)]
    return L, dist

In [9]:
Weights = [0.7, 0.1, 0.1, 0.1]
Weighted_Kameny_consensus(M, Weights)

([array([3, 2, 1, 0])], 0.16666666666666669)

In [44]:
def using_indexed_assignment(x):
    result = np.empty(len(x), dtype=int)
    temp = x.argsort()
    result[temp] = np.arange(len(x))
    return result


def get_ranking(df):
    ranking = df
    for col in df.columns:
        ranking[col] = using_indexed_assignment(df[col])
    return ranking

In [45]:
using_indexed_assignment(np.array([1, 5, 2, 3]))

array([0, 3, 1, 2])

In [48]:
df = pd.DataFrame(
    [[1.2, 1.7, 2], [2, 1.7, 1.2]],
    index=["Algo1", "Algo2"],
    columns=["Test1", "Test2", "Test3"],
)

In [55]:
ranking = get_ranking(df).transpose()

In [56]:
ranking

,Algo1,Algo2
Test1,0,1
Test2,0,1
Test3,1,0
